# Simulating hiring interviews
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for simulating a hiring interview with AI agents. We show 2 different ways of doing this, with and without AI agents:

#### Using question prompts
First we prompt a language model to create an interview dialog between a freelancer candidate and a client hiring for a job, and then prompt the model to pretend it is each of the client and freelancer in the interview and rate the other party's performance in it. We give the model a sample job post and freelancer background (which may or may not be relevant to the job post) to generate the dialog.

#### Using AI agents
In the second iteration we construct AI agents representing the client and freelancer for each of the dialog, tell them that they were the participants in the dialogs, and then prompt them to rate each other. This is different from the first experiment where we instructed the model to draft the dialog and then pretend to be either the client or freelancer with knowledge that it had created the dialog. This time, we simply give the dialog to the agents representing the client and freelancers and then instruct them to do the ratings.

### EDSL
EDSL is an open-source library for simulating surveys and experiments with AI agents and language models. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

We start by importing the tools that we will use to simulate the interviews. EDSL comes with a [variety of question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from the model (multiple choice, free text, etc.). Here we will use a free text question to prompt the model to generate dialogs and a linear scale question for eliciting the ratings of the client and freelancer:

In [1]:
from edsl.questions import QuestionFreeText, QuestionLinearScale
from edsl import Scenario, Survey, Agent, Model

import pandas as pd

EDSL works with many popular language models. We can see a list of current available models:

In [2]:
Model.available()

[['01-ai/Yi-34B-Chat', 'deep_infra', 0],
 ['Austism/chronos-hermes-13b-v2', 'deep_infra', 1],
 ['Gryphe/MythoMax-L2-13b', 'deep_infra', 2],
 ['Gryphe/MythoMax-L2-13b-turbo', 'deep_infra', 3],
 ['HuggingFaceH4/zephyr-orpo-141b-A35b-v0.1', 'deep_infra', 4],
 ['Phind/Phind-CodeLlama-34B-v2', 'deep_infra', 5],
 ['bigcode/starcoder2-15b', 'deep_infra', 6],
 ['bigcode/starcoder2-15b-instruct-v0.1', 'deep_infra', 7],
 ['claude-3-haiku-20240307', 'anthropic', 8],
 ['claude-3-opus-20240229', 'anthropic', 9],
 ['claude-3-sonnet-20240229', 'anthropic', 10],
 ['codellama/CodeLlama-34b-Instruct-hf', 'deep_infra', 11],
 ['codellama/CodeLlama-70b-Instruct-hf', 'deep_infra', 12],
 ['cognitivecomputations/dolphin-2.6-mixtral-8x7b', 'deep_infra', 13],
 ['databricks/dbrx-instruct', 'deep_infra', 14],
 ['deepinfra/airoboros-70b', 'deep_infra', 15],
 ['gemini-pro', 'google', 16],
 ['google/codegemma-7b-it', 'deep_infra', 17],
 ['google/gemma-1.1-7b-it', 'deep_infra', 18],
 ['gpt-3.5-turbo', 'openai', 19],


If we do not specify a model to use in simulating survey results, the default model (GPT 4 preview) will be used. Here we specity a model that we will call later on.

In [3]:
model = Model('gpt-4')

Here we import some mock job posts to use for the simulations:

In [4]:
job_posts = [
    {"title": "Content Writer for Health and Wellness Blog", "details": "We are seeking an experienced content writer to produce engaging and informative articles for our health and wellness blog. Topics include nutrition, exercise, mental health, and lifestyle tips. The ideal candidate will have a background in health sciences and a flair for writing. Strong research skills and SEO knowledge are a plus."},
    {"title": "Graphic Designer for E-commerce Website", "details": "Looking for a talented graphic designer to create eye-catching banners, product images, and promotional materials for our online store. The candidate should be proficient in Adobe Creative Suite and have experience in designing for e-commerce platforms. A keen eye for detail and the ability to meet deadlines are essential."},
    {"title": "Mobile App Developer for Fitness Tracking App", "details": "We need a skilled mobile app developer to create a fitness tracking app for both iOS and Android. The app will include features such as activity logging, progress tracking, and social sharing. Experience with React Native or Flutter is preferred. Applicants should provide a portfolio of previous app development projects."},
    {"title": "Virtual Assistant for Real Estate Company", "details": "Our real estate company is looking for a reliable virtual assistant to handle administrative tasks, including scheduling appointments, managing emails, and maintaining client databases. The ideal candidate will have excellent organizational skills, strong communication abilities, and experience with CRM software."},
    {"title": "SEO Specialist for Online Marketing Campaign", "details": "Seeking an SEO specialist to optimize our website and improve search engine rankings. Responsibilities include keyword research, on-page optimization, link building, and performance tracking. Candidates should have a proven track record in SEO and familiarity with tools like Google Analytics and SEMrush."},
    {"title": "Social Media Manager for Fashion Brand", "details": "We are in need of a creative social media manager to develop and execute social media strategies for our fashion brand. The role involves content creation, scheduling posts, engaging with followers, and analyzing performance metrics. Experience with Instagram, Facebook, and TikTok is required."},
    {"title": "Web Developer for Custom WordPress Site", "details": "Looking for an experienced web developer to build a custom WordPress site for our small business. The project includes creating a responsive design, integrating plugins, and ensuring site security. Knowledge of HTML, CSS, JavaScript, and PHP is necessary. Prior experience with WordPress development is essential."},
    {"title": "Customer Support Representative for SaaS Company", "details": "Our SaaS company is hiring a customer support representative to assist users with technical issues and provide product guidance. The candidate should have excellent communication skills, a patient demeanor, and the ability to troubleshoot problems effectively. Experience in customer service and familiarity with SaaS products is preferred."},
    {"title": "Video Editor for YouTube Channel", "details": "We are seeking a skilled video editor to join our team and edit videos for our YouTube channel. The role includes cutting footage, adding effects, and ensuring high-quality production. Proficiency in video editing software like Adobe Premiere Pro or Final Cut Pro is required. A creative approach and attention to detail are key."},
    {"title": "Data Analyst for Market Research Project", "details": "Looking for a data analyst to conduct market research and provide insights based on data analysis. Responsibilities include collecting data, analyzing trends, and preparing reports. Candidates should have strong analytical skills, experience with statistical software (e.g., R, Python), and the ability to present findings clearly."}
]

Here we create some example freelancers applying to jobs:

In [5]:
freelancers = [
    {"name": "Jane Doe", "role": "Content Writer", "experience": "An experienced content writer with a background in health sciences. Has worked with various health and wellness blogs, producing engaging and informative articles on topics like nutrition, exercise, and mental health. Strong research skills and SEO knowledge have consistently helped clients improve their online visibility."},
    {"name": "John Smith", "role": "Graphic Designer", "experience": "A talented graphic designer with a degree in Fine Arts and over five years of experience in e-commerce design. Proficient in Adobe Creative Suite and has a keen eye for detail, having created eye-catching banners and product images for multiple online stores. Known for creativity and ability to meet tight deadlines."},
    {"name": "Sarah Johnson", "role": "Mobile App Developer", "experience": "A skilled mobile app developer with expertise in both iOS and Android platforms. With a background in computer science, has developed several fitness tracking apps using React Native and Flutter. Portfolio showcases ability to integrate features like activity logging, progress tracking, and social sharing."},
    {"name": "Michael Brown", "role": "Virtual Assistant", "experience": "Over six years of experience as a virtual assistant in the real estate industry. Excels in handling administrative tasks such as scheduling appointments, managing emails, and maintaining client databases. Organizational skills and strong communication abilities make an asset to any team."},
    {"name": "Emily Green", "role": "SEO Specialist", "experience": "An SEO specialist with a proven track record in optimizing websites and improving search engine rankings. Holds a degree in marketing and has extensive experience with tools like Google Analytics and SEMrush. Known for expertise in keyword research, on-page optimization, and link building."},
    {"name": "David Lee", "role": "Social Media Manager", "experience": "A creative social media manager with a background in fashion marketing. Has successfully developed and executed social media strategies for various fashion brands, focusing on content creation, scheduling posts, and engaging with followers. Experience with Instagram, Facebook, and TikTok has helped clients achieve significant growth in their online presence."},
    {"name": "Sophia Martinez", "role": "Web Developer", "experience": "An experienced web developer specializing in custom WordPress sites. With a background in computer science, has built numerous websites for small businesses, ensuring responsive design, plugin integration, and site security. Technical skills in HTML, CSS, JavaScript, and PHP are complemented by attention to detail."},
    {"name": "James Wilson", "role": "Customer Support Representative", "experience": "A decade of experience in customer support, particularly within the SaaS industry. Known for excellent communication skills and patient demeanor, which help effectively assist users with technical issues and product guidance. Familiarity with various SaaS products enables efficient problem troubleshooting."},
    {"name": "Olivia Davis", "role": "Video Editor", "experience": "A skilled video editor with a degree in film production and over seven years of experience editing for YouTube channels. Proficient in Adobe Premiere Pro and Final Cut Pro, work includes cutting footage, adding effects, and ensuring high-quality production. Creative approach and attention to detail make videos stand out."},
    {"name": "Alexander Miller", "role": "Data Analyst", "experience": "A data analyst with a background in statistics and market research. Extensive experience in collecting data, analyzing trends, and preparing insightful reports. Proficiency with statistical software like R and Python, along with the ability to present findings clearly, has helped businesses make data-driven decisions."}
]

General personas and instructions for the client and freelancer agents:

## Question prompts
In the first iteration of this experiment we create questions prompting the model to generate an interview dialog, and then prompt the model to pretend it is each of the client and freelancer in the interview and rate the other party's performance in it. We create {{ placeholders }} in the initial question where we will insert the job post and freelancer background for the model to use in generating a dialog. (Note that we allow the freelancer backgrounds to be relevant or irrelevant to the job post to elicit interesting ratings):

In [6]:
q_dialog = QuestionFreeText(
    question_name = "dialog",
    question_text = """Draft a short realistic dialog of a client interviewing 
    a freelancer for a job on an online labor marketplace (the freelancer may 
    or may not be qualified for the job): 
    Job post: {{ job_details }}
    Freelancer experience: {{ freelancer_experience }}"""
)

q_freelancer_performance = QuestionLinearScale(
    question_name = "freelancer_performance",
    question_text = """Now imagine that you are the client who conducted the
    interview. Rate the freelancer's performance on a scale from 0 to 10.""",
    question_options = [0,1,2,3,4,5,6,7,8,9,10],
    option_labels = {0: "Terrible", 10: "Excellent"}
)

q_client_performance = QuestionLinearScale(
    question_name = "client_performance",
    question_text = """Now imagine that you are the freelancer who was 
    interviewed. Rate the client's performance on a scale from 0 to 10.""",
    question_options = [0,1,2,3,4,5,6,7,8,9,10],
    option_labels = {0: "Terrible", 10: "Excellent"}
)

### Survey memories
We combine the questions into a survey and then apply rules giving the model a memory of the dialog question and response when answering each of the subsequent questions:

In [7]:
survey = (Survey([q_dialog, q_freelancer_performance, q_client_performance])
          .add_targeted_memory(q_freelancer_performance, q_dialog)
          .add_targeted_memory(q_client_performance, q_dialog)
         )

### Scenarios
We use scenarios to input the job post and freelancer information in the "dialog" question text, so that we can repeat variations of it at scale. Not that we also include other details (metadata) about the freelancers that we want to keep together with the inputs and results when we analyze the results. This will conveniently create additional columns of those details in the results without having to pass them all to the model in the question texts:

In [8]:
scenarios = [Scenario({"job_title": j["title"], 
                       "job_details": j["details"], 
                       "freelancer_name": f["name"],
                       "freelancer_role": f["role"],
                       "freelancer_experience": f["experience"]})
            for j in job_posts for f in freelancers]

We can see how many were created and inspect them:

In [9]:
# scenarios
len(scenarios)

100

### Running the survey
Finally, we run the survey with some of the scenarios to generate results, We can optionally include a progress bar:

In [10]:
results = survey.by(scenarios[0:5]).run(progress_bar=True)

Output()

Exceptions were raised in the following interviews: [3]
The returned results have a ".show_exceptions()" attribute e.g., 

>>> results = suvey.by(agents).by(scenarios).run() 
>>> results.show_exceptions()

Exceptions details are available here: 

>>> from edsl import shared_globals
>>> shared_globals['edsl_runner_exceptions'].show_exceptions()

For more details see documentation: https://docs.expectedparrot.com/en/latest/exceptions.html

There is also a ".failed_jobs" attribute that contains the interviews that failed, as a job. 
This can be used to re-run the failed interviews.


>>> results.failed_jobs.run()




In [11]:
# results.show_exceptions()

### Inspecting results
EDSL comes with built-in tools for analyzing results. For example, we can see a list of all the components of the results and then select some to print in a table:

In [12]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.client_performance',
 'answer.dialog',
 'answer.freelancer_performance',
 'comment.client_performance_comment',
 'comment.dialog_comment',
 'comment.freelancer_performance_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.client_performance_system_prompt',
 'prompt.client_performance_user_prompt',
 'prompt.dialog_system_prompt',
 'prompt.dialog_user_prompt',
 'prompt.freelancer_performance_system_prompt',
 'prompt.freelancer_performance_user_prompt',
 'question_options.client_performance_question_options',
 'question_options.dialog_question_options',
 'question_options.freelancer_performance_question_options',
 'question_text.client_performance_question_text',
 'question_text.dialog_question_text',
 'question_text.freelancer_performance_question_text',
 'question_typ

Note that each question type (other than free text) automatically includes an additional "_comment" field where the model can add any commentary about its response to the question. This can be useful for qualitative analysis and running follow-on questions.

In [13]:
(results
 .select("job_title", "freelancer_role", "client_performance", "client_performance_comment", "freelancer_performance", "freelancer_performance_comment")
 .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ scenario         ┃ scenario         ┃ answer           ┃ answer           ┃ comment          ┃ comment          ┃
┃ .job_title       ┃ .freelancer_role ┃ .client_perform… ┃ .freelancer_per… ┃ .client_perform… ┃ .freelancer_per… ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Content Writer   │ Content Writer   │ 8                │ 9                │ The client was   │ The freelancer   │
│ for Health and   │                  │                  │                  │ clear about the  │ demonstrated a   │
│ Wellness Blog    │                  │                  │                  │ job requirements │ strong           │
│                  │                  │                  │                  │ and asked        │ understanding of │
│                  │                  │                  │                  │ relevant         │ the job          │
│                  │                  │                  │                  │ questions that   │ requirements,    │
│                  │                  │                  │                  │ allowed me to    │ showcased        │
│                  │                  │                  │                  │ demonstrate my   │ relevant         │
│                  │                  │                  │                  │ qualifications   │ experience and   │
│                  │                  │                  │                  │ and experience.  │ knowledge in     │
│                  │                  │                  │                  │ They also        │ health sciences, │
│                  │                  │                  │                  │ inquired about   │ and exhibited    │
│                  │                  │                  │                  │ my SEO and       │ good             │
│                  │                  │                  │                  │ research skills, │ communication    │
│                  │                  │                  │                  │ which are        │ skills. They     │
│                  │                  │                  │                  │ critical for the │ also have a      │
│                  │                  │                  │                  │ role. The client │ solid grasp of   │
│                  │                  │                  │                  │ listened and     │ SEO and          │
│                  │                  │                  │                  │ responded        │ research, which  │
│                  │                  │                  │                  │ professionally,  │ are crucial for  │
│                  │                  │                  │                  │ creating a       │ the role. The    │
│                  │                  │                  │                  │ positive         │ only reason for  │
│                  │                  │                  │                  │ interview        │ not giving a     │
│                  │                  │                  │                  │ experience.      │ full score is    │
│                  │                  │                  │                  │ However, there's │ because the      │
│                  │                  │                  │                  │ always room for  │ performance in   │
│                  │                  │                  │                  │ improvement,     │ an actual work   │
│                  │                  │                  │                  │ possibly in      │ setting has yet  │
│                  │                  │                  │                  │ providing more   │ to be observed.  │
│                  │                  │                  │                  │ details about    │                  │
│                  │                  │                 

## Using AI Agents
In the second iteration of this experiment we construct agents representing a client and freelancers, tell them that they were the participants in the dialogs, and then prompt them to rate each other. This is different from the first experiment where we instructed the model to draft the dialog and then pretend to be either the client or freelancer with knowledge that it had created the dialog. This time, we simply give the dialog to the agents representing the client and freelancers and then instruct them to do the ratings.

### Accessing the dialogs
We start by transforming the results of the prior survey into a dataframe, so that we can easily access the scenarios (job posts and freelancer details) together with the dialogs that were generated. These will become the inputs to our new survey questions:

In [14]:
df = results.to_pandas(remove_prefix=True) # We can remove the 'agent.', 'answer.', etc. prefixes for convenience

We can inspect the dataframe and select the columns that we need:

In [15]:
df[["job_title", "job_details", "freelancer_role", "freelancer_experience", "dialog"]]

,job_title,job_details,freelancer_role,freelancer_experience,dialog
0,Content Writer for Health and Wellness Blog,We are seeking an experienced content writer t...,Content Writer,An experienced content writer with a backgroun...,"Client: Hi there, thanks for applying to our c..."
1,Content Writer for Health and Wellness Blog,We are seeking an experienced content writer t...,Graphic Designer,A talented graphic designer with a degree in F...,"Client: Hi there, thanks for connecting with m..."
2,Content Writer for Health and Wellness Blog,We are seeking an experienced content writer t...,Mobile App Developer,A skilled mobile app developer with expertise ...,"Client: Hello, thank you for taking the time t..."
3,Content Writer for Health and Wellness Blog,We are seeking an experienced content writer t...,SEO Specialist,An SEO specialist with a proven track record i...,"Client: Hello, thank you for joining me today...."
4,Content Writer for Health and Wellness Blog,We are seeking an experienced content writer t...,Virtual Assistant,Over six years of experience as a virtual assi...,"Client: Hi, thanks for connecting. I see you'v..."


Next we create a client agent and freelancer agent for each combination of job post, freelancer and dialog in the original results, prompt them to rate the other party in the dialog, and compile the results:

In [16]:
# Base personas for the agents
client_persona = "You are a client interviewing freelancers for a job post on an online labor marketplace."
freelancer_persona = "You are a freelancer applying for a job on an online labor marketplace."

# The sets of ratings results
client_ratings_by_freelancers = None
freelancer_ratings_by_clients = None

# Iterate through the job post/freelancer/dialog results to create agents and administer the ratings questions
for index, row in df.iterrows():
    job_title = row["job_title"]
    job_details = row["job_details"]
    
    # Each client agent has the base persona and instructions noting the job post title and details for the dialog
    client = Agent(
        traits={"persona": client_persona},
        instruction=f"""You are interviewing a freelancer for a job post: {job_title}
        Description: {job_details}"""
    )

    # Each freelancer agent has the base persona and freelancer details for the dialog
    freelancer = Agent(
        traits={"persona": freelancer_persona, 
                "freelancer_name": row["freelancer_name"],
                "freelancer_role": row["freelancer_role"], 
                "freelancer_experience": row["freelancer_experience"]}
    )

    # Question prompting the freelancer to rate the client 
    q_client_rating = QuestionLinearScale(
        question_name = "client_rating",
        question_text = """You have just been interviewed for a job.
        Review the transcript of the interview and then rate the client's  
        performance on a scale from 0 to 10: {{ dialog }}""",
        question_options = [0,1,2,3,4,5,6,7,8,9,10],
        option_labels = {0: "Terrible", 10: "Fantastic"}
    )

    # Question prompting the client to rate the freelancer:
    q_freelancer_rating = QuestionLinearScale(
        question_name = "freelancer_rating",
        question_text = """You have just interviewed a freelancer for a job.
        Review the transcript of the interview and then rate the freelancer's  
        performance on a scale from 0 to 10: {{ dialog }}""",
        question_options = [0,1,2,3,4,5,6,7,8,9,10],
        option_labels = {0: "Terrible", 10: "Fantastic"}
    )

    # Parameterize the questions with the dialogs and add metadata to include with results
    scenario = Scenario({"job_title": row["job_title"],  
                         "job_details": row["job_details"],
                         "freelancer_name": row["freelancer_name"],
                         "freelancer_role": row["freelancer_role"],
                         "freelancer_experience": row["freelancer_experience"],
                         "dialog": row["dialog"]})
    
    # Generating the client rating by freelancer and adding it to the results
    if client_ratings_by_freelancers == None:
        client_ratings_by_freelancers = q_client_rating.by(freelancer).by(scenario).run()
    else:
        new_results = q_client_rating.by(freelancer).by(scenario).run()
        client_ratings_by_freelancers = client_ratings_by_freelancers + new_results

    # Generating the freelancer rating by client and adding it to the results
    if freelancer_ratings_by_clients == None:
        freelancer_ratings_by_clients = q_freelancer_rating.by(client).by(scenario).run()
    else:
        new_results = q_freelancer_rating.by(client).by(scenario).run()
        freelancer_ratings_by_clients = freelancer_ratings_by_clients + new_results

We can use dataframes to compare the results side-by-side for each dialog:

In [17]:
df_client = client_ratings_by_freelancers.to_pandas(remove_prefix=True) 
df_freelancer = freelancer_ratings_by_clients.to_pandas(remove_prefix=True)

In [18]:
common_columns = ["job_title", "job_details", "freelancer_name", "freelancer_role", "freelancer_experience", "dialog"]

In [19]:
df_combined = pd.merge(df_client, df_freelancer, how="inner", left_on=common_columns, right_on=common_columns)

In [20]:
df_combined.columns

Index(['agent_instruction_x', 'agent_name_x', 'client_rating',
       'client_rating_comment', 'client_rating_question_options',
       'client_rating_question_text', 'client_rating_question_type',
       'client_rating_raw_model_response', 'client_rating_system_prompt',
       'client_rating_user_prompt', 'dialog', 'edsl_class_name_x',
       'edsl_version_x', 'freelancer_experience', 'freelancer_name',
       'freelancer_role', 'frequency_penalty_x', 'iteration_x', 'job_details',
       'job_title', 'logprobs_x', 'max_tokens_x', 'model_x', 'persona_x',
       'presence_penalty_x', 'temperature_x', 'top_logprobs_x', 'top_p_x',
       'agent_instruction_y', 'agent_name_y', 'edsl_class_name_y',
       'edsl_version_y', 'freelancer_rating', 'freelancer_rating_comment',
       'freelancer_rating_question_options', 'freelancer_rating_question_text',
       'freelancer_rating_question_type',
       'freelancer_rating_raw_model_response',
       'freelancer_rating_system_prompt', 'freelancer

In [21]:
df_combined[["job_title", "freelancer_role", "client_rating", "client_rating_comment", "freelancer_rating", "freelancer_rating_comment"]]

,job_title,freelancer_role,client_rating,client_rating_comment,freelancer_rating,freelancer_rating_comment
0,Content Writer for Health and Wellness Blog,Content Writer,9,The client conducted a thorough and profession...,9,The freelancer demonstrated a strong backgroun...
1,Content Writer for Health and Wellness Blog,Graphic Designer,7,The client was clear about the job requirement...,6,The freelancer demonstrated a positive attitud...
2,Content Writer for Health and Wellness Blog,Mobile App Developer,7,The client conducted a professional and focuse...,5,The freelancer has relevant experience in the ...
3,Content Writer for Health and Wellness Blog,SEO Specialist,8,"The client conducted a professional interview,...",8,The freelancer demonstrated a good understandi...
4,Content Writer for Health and Wellness Blog,Virtual Assistant,8,The client conducted a professional and focuse...,5,The freelancer has demonstrated transferable s...


deferred acceptance algorithm